In [1]:
from pyspark.sql import functions as sf, DataFrame, SparkSession

from feature_logic.main import create_features
from feature_logic.feature_groups.test_group import LocationFeatures

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 15:40:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = spark.createDataFrame([dict(id="a"), dict(id="b"), dict(id="c")])

location_data = spark.createDataFrame([
    dict(id="a", postcode="1234AB"),
    dict(id="b", postcode="1234BC"),
    dict(id="c", postcode=""),
])
location_data.write.mode("overwrite").parquet("data/location_data.parquet")


In [7]:
features = create_features(
    data=data,
    key="id", 
    feature_groups=[
        LocationFeatures(spark=spark, features=["postcode"]),
    ])

In [8]:
features.toPandas()

,id,postcode
0,a,1234AB
1,b,1234BC
2,c,


In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:

wow = spark.read.parquet("wow.parquet")
wow.limit(10).toPandas()

In [ ]:
wow.select("zone").distinct().collect()

In [ ]:
(
    wow
    .withColumn("durotar_flag", sf.when(sf.col("zone") == " Durotar", 1).otherwise(0))
    .groupby("avatarId")
    .agg(
        sf.count("*").alias("total_count"),
        sf.sum("durotar_flag").alias("durotar_count")
    )
    .withColumn("durotar_prob", sf.col("durotar_count") / sf.col("total_count"))
    .show()
)